In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')
data = np.load('/content/drive/My Drive/Github/mtc-device-activation/data/activity-models/e1-general-model.npz', allow_pickle=True)
all_DeviceLocations = data['all_DeviceLocations']
num_samples = data['num_samples']
device_count = data['device_count']
sparsity = data['sparsity']

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e1-communication-simulation-theta.npz", allow_pickle=True)
all_theta = data["all_theta"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e1-communication-simulation-gamma.npz", allow_pickle=True)
all_gamma = data["all_gamma"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e1-communication-simulation-z.npz", allow_pickle=True)
all_z = data["all_z"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e1-communication-simulation-noise.npz", allow_pickle=True)
all_noise = data["all_noise"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e1-communication-simulation-received-signal.npz", allow_pickle=True)
all_received_signal = data["all_received_signal"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e1-communication-simulation-general.npz", allow_pickle=True)
snr_db_array = data["snr_db_array"]
pilot_length_array = data["pilot_length_array"]
M = data["M"]

pilot_length_index = 1
pilot_length=pilot_length_array[pilot_length_index]

snr_db_index = 2
snr_db=snr_db_array[snr_db_index]

alpha_array = [0, 0.5, 1.0, 1.5, 2, 2.5, 3]
beta = 0.2

Mounted at /content/drive


In [ ]:
sample_range = range(600, 800)

Correlation-Aware Sparse Bayesian Learning (CA-SBL)

In [ ]:
def casbl_algorithm(A, y, N, sig2e, Tau_p, loc, alpha=0.1, beta=0.2, kappa=5.725, D=20, max_iter=500, stopping_criterion=1e-4):
    """Correlation Aware Sparse Bayesian Learning (CASBL) for Multiple Measurement Vectors (MMV)."""

    M = y.shape[1]  # Number of antennas

    # Initialize Gamma (covariance matrix of the signal)
    Gamma = np.eye(N) * 0.1  # Initial guess

    if kappa == 0:
        correlation_matrix = np.eye(N)  # Identity matrix when no correlation
    else:
        # Compute pairwise distances (N x N matrix)
        distance_matrix = np.linalg.norm(loc[:, np.newaxis, :] - loc[np.newaxis, :, :], axis=2)

        # Compute exponential decay matrix
        correlation_matrix = np.maximum((np.exp(-distance_matrix / kappa) - np.exp(-D / kappa)) / (1 - np.exp(-D / kappa)),0)

    # Loop over the maximum number of iterations
    for t in range(max_iter):
        # E-Step: Compute the mean (mu_x) and covariance (Sigma_x) of x
        Sigma_y = A @ Gamma @ A.conj().T + sig2e * np.eye(Tau_p)
        Sigma_y_inv = np.linalg.inv(Sigma_y)
        Sigma_x = Gamma - Gamma @ A.conj().T @ Sigma_y_inv @ A @ Gamma
        mu_x = Gamma @ A.conj().T @ Sigma_y_inv @ y

        # Compute Q
        Q = (np.linalg.norm(mu_x, axis=1) ** 2) / M + np.diag(Sigma_x)

        # Compute P
        P = 2 * alpha * (beta - correlation_matrix) @ np.diag(Gamma)
        P = np.maximum(P, 1e-8)

        # Compute gamma update
        gamma_new = (np.sqrt(1 + 4 * P * Q) - 1) / (2 * P)

        # Ensure non-negativity (numerical stability)
        gamma_new = np.clip(np.real(gamma_new), 0, 1)

        # Update Gamma
        Gamma_old = np.diag(Gamma)
        Gamma = np.diagflat(gamma_new)

        # Check for convergence
        if np.linalg.norm(Gamma_old - gamma_new) < stopping_criterion:
            print(f"Converged after {t+1} iterations")
            break

    Gamma = np.where(Gamma < 0.05, 0, 1)

    Sigma_y = A @ Gamma @ A.conj().T + sig2e * np.eye(Tau_p)
    Sigma_y_inv = np.linalg.inv(Sigma_y)
    Sigma_x = Gamma - Gamma @ A.conj().T @ Sigma_y_inv @ A @ Gamma
    mu_x = Gamma @ A.conj().T @ Sigma_y_inv @ y

    return mu_x, np.diag(Gamma)  # Final estimates


In [ ]:
# Placeholder arrays for casbl results
z_casbl = np.zeros((num_samples, len(alpha_array), device_count, M), dtype=np.complex128)  # casbl results
gamma_casbl = np.zeros((num_samples, len(alpha_array), device_count), dtype=bool)  # casbl gamma values
hit_rate_casbl = np.zeros((num_samples, len(alpha_array)))  # Hit rate for casbl
support_recovery_rate_casbl = np.zeros((num_samples, len(alpha_array)))  # Miss detection rate for casbl
false_alarm_rate_casbl = np.zeros((num_samples, len(alpha_array)))  # False alarm rate for casbl
norm_mse_casbl = np.zeros((num_samples, len(alpha_array)))  # Norm MSE for casbl

In [ ]:
# Loop through each sample with an outer progress bar
for sample_index in tqdm(sample_range, desc="Processing Samples", position=0):
    theta = all_theta[sample_index, pilot_length_index, :pilot_length]
    gamma = all_gamma[sample_index]  # True gamma values (ground truth)
    loc = all_DeviceLocations[sample_index]

    received_signal = all_received_signal[sample_index, pilot_length_index, snr_db_index, :pilot_length]
    z = all_z[sample_index]
    snr_db = snr_db_array[snr_db_index]
    snr = 10 ** (snr_db / 10)
    signal_power = np.mean(np.abs(np.matmul(theta, z))**2)
    noise_power = signal_power / snr

    # Loop over each SNR level with an inner progress bar
    for alpha_index in tqdm(range(len(alpha_array)), desc="Processing alpha Levels", leave=False, position=1):
        alpha = alpha_array[alpha_index]

        # Apply casbl algorithm for the current sample and SNR level
        z_result, gamma_result = casbl_algorithm(theta, received_signal, device_count, noise_power, pilot_length, loc, alpha=alpha, beta=beta)

        # Store the results
        z_result[gamma_result <= 0.05] = 0
        z_casbl[sample_index, alpha_index, :, :] = z_result
        gamma_result_bool =  gamma_result > 0.05
        gamma_casbl[sample_index, alpha_index, :] = gamma_result_bool

        # Avoid division by zero in hit/miss/false alarm rate calculations
        active_devices = np.sum(gamma == 1)  # True active devices
        inactive_devices = np.sum(gamma == 0)  # True inactive devices

        # Calculate the hit rate for casbl
        hit_rate_casbl[sample_index, alpha_index] = (
            100 * np.sum((gamma_result_bool == 1) & (gamma == 1)) / active_devices if active_devices > 0 else 0
        )

        # Calculate the support recovery rate for casbl
        support_recovery_rate_casbl[sample_index, alpha_index] = (
            np.sum((gamma_result_bool ==1) & (gamma == 1))
            /
            (np.sum((gamma_result_bool == 1) & (gamma == 0)) + np.sum((gamma_result_bool == 0) & (gamma == 1)) + sparsity)
        )

        # Calculate the false alarm rate for casbl
        false_alarm_rate_casbl[sample_index, alpha_index] = (
            100 * np.sum((gamma_result_bool == 1) & (gamma == 0)) / inactive_devices if inactive_devices > 0 else 0
        )

        # Calculate the normalized MSE for casbl
        norm_mse_casbl[sample_index, alpha_index] = (
            np.linalg.norm(z[gamma == 1] - z_result[gamma == 1])**2 / np.linalg.norm(z[gamma == 1])**2
        )



Processing alpha Levels:  57%|█████▋    | 4/7 [01:09<00:53, 17.75s/it]

Converged after 493 iterations



Processing alpha Levels:  57%|█████▋    | 4/7 [01:09<00:51, 17.04s/it]

Converged after 446 iterations



Processing alpha Levels:  71%|███████▏  | 5/7 [01:24<00:32, 16.10s/it]

...

Converged after 462 iterations



Processing alpha Levels:  86%|████████▌ | 6/7 [01:41<00:16, 16.37s/it]

Converged after 448 iterations



Processing Samples: 100%|██████████| 200/200 [6:41:45<00:00, 120.53s/it]

Converged after 450 iterations


In [ ]:
# Compute the average across all samples for each SNR level
avg_hit_rate_casbl = np.mean(hit_rate_casbl, axis=0)  # Shape: (len(snr_db_array),)
avg_support_recovery_rate_casbl = np.mean(support_recovery_rate_casbl, axis=0)  # Shape: (len(snr_db_array),)
avg_false_alarm_rate_casbl = np.mean(false_alarm_rate_casbl, axis=0)  # Shape: (len(snr_db_array),)
avg_norm_mse_casbl = np.mean(norm_mse_casbl, axis=0)  # Shape: (len(snr_db_array),)

# Print or log the results
print("Average Hit Rate across samples per alpha level:", avg_hit_rate_casbl)
print("Average Support Recovery Rate across samples per alpha level:", avg_support_recovery_rate_casbl)
print("Average False Alarm Rate across samples per alpha level:", avg_false_alarm_rate_casbl)
print("Average Normalized MSE across samples per alpha level:", avg_norm_mse_casbl)

Average Hit Rate across samples per alpha level: [16.17 18.3  18.66 18.76 18.72 18.57 18.39]
Average Support Recovery Rate across samples per alpha level: [0.09495463 0.13809958 0.15150986 0.15423519 0.15331626 0.14938485
 0.1448602 ]
Average False Alarm Rate across samples per alpha level: [0.33358974 0.16205128 0.11333333 0.10692308 0.10846154 0.12076923
 0.13384615]
Average Normalized MSE across samples per alpha level: [0.07355496 0.04614797 0.03842473 0.03678107 0.03752731 0.04042707
 0.04234583]


In [ ]:
np.savez_compressed('/content/drive/My Drive/Github/mtc-device-activation/data/results/e1-results-1000-casbl-anc-alpha-all-beta-0-2-snr-db-12-pilot-length-15-set-4.npz',
                    z_casbl=z_casbl,
                    gamma_casbl=gamma_casbl,
                    hit_rate_casbl=hit_rate_casbl,
                    support_recovery_rate_casbl=support_recovery_rate_casbl,
                    false_alarm_rate_casbl=false_alarm_rate_casbl,
                    norm_mse_casbl=norm_mse_casbl,
                    avg_hit_rate_casbl=avg_hit_rate_casbl,
                    avg_support_recovery_rate_casbl=avg_support_recovery_rate_casbl,
                    avg_false_alarm_rate_casbl=avg_false_alarm_rate_casbl,
                    avg_norm_mse_casbl=avg_norm_mse_casbl,
                    pilot_length=pilot_length,
                    snr_db=snr_db,
                    beta = beta,
                    alpha_array=alpha_array)

print("All results have been saved")

All results have been saved
